# Carga de los conjuntos de datos desde Drive

Librerias necesarias para acceder a Drive

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Autenticación de Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Importacón de los archivos zip ubicados en 

Para replicar la experimentación se debe colocar ambos archivos zip en su propia unidad Drive

In [ ]:
fid = drive.ListFile({'q':"title='KS.zip'"}).GetList()[0]['id']
m = drive.CreateFile({'id': fid})
m.GetContentFile('KS.zip')

Extracción de los audios

In [ ]:
!unzip KS.zip -d ''

Archive:  KS.zip
   creating: VC Bien Pronunciadas/
  inflating: VC Bien Pronunciadas/253_5_BM_alfombra.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_auto.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_bufanda.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_caperusita.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_edificio.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_foca.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_guante.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_helicoptero.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_indio.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_mariposa.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_pantalon.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_platano.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_plato.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_telefono.wav  
  inflating: VC Bien Pronunciadas/253_5_BM_tren.wav  
  inflating: VC Bien Pronunciadas/253_5_CA_alfombra.wav  
  inflating: VC Bien Pronunciadas/253

# 3.Pre-procesamiento de los datos

In [ ]:
import os
import librosa
import skimage.io
from skimage.transform import resize
from skimage import img_as_ubyte
from skimage.color import gray2rgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Función para escalar los datos detro del rango 8-bit

In [ ]:
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

Función para obtener el espectograma Mel de los audios

In [ ]:
def spectrogram_image(y, sr):
    mels = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    #mels = np.log(mels + 1e-9)

    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0)
    img = 255-img

    return img

Recorrer los archivos de audio para obtener sus espectogramas y almacenarlos en el arreglo "specgrams"

In [ ]:
acurracies = []
precisions = []
recalls = []

In [ ]:
pitch = ['253', '255', '309']
sr = ['5', '7']
names = ['BM', 'CA', 'DJ', 'DM', 'EC', 'EV', 'IL', 'JE', 'KM', 'LZ', 'MB', 'OF', 'VL', 'WH']

In [ ]:
lista = []
for i in pitch:
  for j in sr:
    for name in names:
      lista.append(i + '_' + j + '_' + name)

In [ ]:
lista_test = []

In [ ]:
import random
while len(lista_test) < 25:
  num = random.randint(0, 83)
  if lista[num] not in lista_test:
    lista_test.append(lista[num])

In [ ]:
lista_test

['253_5_LZ',
 '253_7_BM',
 '309_7_DJ',
 '309_7_JE',
 '255_7_BM',
 '309_5_BM',
 '255_5_EV',
 '255_7_OF',
 '253_7_JE',
 '255_5_VL',
 '255_5_CA',
 '253_5_BM',
 '255_5_DJ',
 '253_5_DJ',
 '253_7_EC',
 '253_5_EV',
 '253_7_CA',
 '309_5_EC',
 '309_5_LZ',
 '309_7_VL',
 '309_7_OF',
 '309_5_DJ',
 '253_5_OF',
 '309_7_CA',
 '309_5_JE']

In [ ]:
specgrams = []
specgrams2 = []
archivos = os.listdir('VC Mal Pronunciadas')
archivos.sort()
for audio in archivos:
  array = audio.split('_')
  temp = audio[:8]
  if audio.endswith('.wav') and array[2] != 'RR':
    y, sr = librosa.load('VC Mal Pronunciadas/' + audio)
    image = spectrogram_image(y, sr)
    image_resized = resize(image, (224, 224), anti_aliasing=True)
    if temp in lista_test:
      specgrams2.append([image_resized, 0])
    else:
      specgrams.append([image_resized, 0])

archivos = os.listdir('VC Bien Pronunciadas')
archivos.sort()
for audio in archivos:
  array = audio.split('_')
  temp = audio[:8]
  if audio.endswith('.wav') and array[2] != 'RR':
    y, sr = librosa.load('VC Bien Pronunciadas/' + audio)
    image = spectrogram_image(y, sr)
    image_resized = resize(image, (224, 224), anti_aliasing=True)
    if temp in lista_test:
      specgrams2.append([image_resized, 1])
    else:
      specgrams.append([image_resized, 1])

In [ ]:
df = pd.DataFrame(specgrams)
df2 = pd.DataFrame(specgrams2)
df = df.sample(frac=1).reset_index(drop=True)
df2 = df2.sample(frac=1).reset_index(drop=True)

#10 Epochs

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.optim as optim

In [ ]:
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

AlexNet_model.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
AlexNet_model.classifier[4] = nn.Linear(4096,1024)
AlexNet_model.classifier[6] = nn.Linear(1024,2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
AlexNet_model.to(device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


cpu


AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
def toDataLoader(data):
  input = torch.tensor(list(data[0].to_numpy()))
  target = torch.tensor(list(data[1])).type(torch.LongTensor)
  tensor = data_utils.TensorDataset(input, target) 
  loader = data_utils.DataLoader(dataset = tensor, batch_size=4, shuffle=True)
  return loader

In [ ]:
#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(Adam)
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
#train_loader = toDataLoader(df)

for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # obtener los inputs; "data" es una lista de [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()

        inputs = inputs.unsqueeze(1)

        # forward + backward + optimize
        output = AlexNet_model(inputs.float())
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()

        # imprimir estadísticas
        running_loss += loss.item()
        if i % 150 == 149:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 150))
            running_loss = 0.0

print('Finished Training of AlexNet')

[1,   150] loss: 0.707
[1,   300] loss: 0.675
[1,   450] loss: 0.677
[2,   150] loss: 0.671
[2,   300] loss: 0.670
[2,   450] loss: 0.658
[3,   150] loss: 0.646
[3,   300] loss: 0.662
[3,   450] loss: 0.621
[4,   150] loss: 0.606
[4,   300] loss: 0.549
[4,   450] loss: 0.569
[5,   150] loss: 0.566
[5,   300] loss: 0.568
[5,   450] loss: 0.632
[6,   150] loss: 0.569
[6,   300] loss: 0.501
[6,   450] loss: 0.519
[7,   150] loss: 0.518
[7,   300] loss: 0.439
[7,   450] loss: 0.497
[8,   150] loss: 0.453
[8,   300] loss: 0.542
[8,   450] loss: 0.445


In [ ]:
test_loader = toDataLoader(df2)

#Testing Accuracy
correct = 0
total = 0
TP = 0
TN = 0
FN = 0
FP = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        images = images.unsqueeze(1)
        outputs = AlexNet_model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)

        for j in range(len(predicted)):
            if predicted[j] == labels[j] and predicted[j] == 0:
                TP += 1
            if predicted[j] == labels[j] and predicted[j] == 1:
                TN += 1
            if predicted[j] != labels[j] and predicted[j] == 0:
                FP += 1
            if predicted[j] != labels[j] and predicted[j] == 1:
                FN += 1
        
        correct += (predicted == labels).sum().item()

print('%d\t%d' % (TP, FN))
print('%d\t%d' % (FP, TN))
print('Accuracy test: %d%%' % (100 *(TP + TN) / (TP + TN + FN + FP)))
#acurracies.append(100 *(TP + TN) / (TP + TN + FN + FP))
print('Precision test: %d%%' % (100 * TP / (TP + FP)))
#precisions.append(100 * TP / (TP + FP))
print('Recall test: %d%%' % (100 * TP / (TP + FN)))
#recalls.append(100 * TP / (TP + FN))

528	20
134	241
Accuracy test: 83%
Precision test: 79%
Recall test: 96%


In [ ]:
torch.save(AlexNet_model, 'model.pkl')

In [ ]:
print(acurracies)
print(precisions)
print(recalls)

# 20 Epochs

In [ ]:
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
AlexNet_model1 = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

#Descripción del modelo
AlexNet_model1.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
#Modificación de la primera capa convolucional
AlexNet_model1.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

#Modificación de la penultima capa completamente conectada
AlexNet_model1.classifier[4] = nn.Linear(4096,1024)

#Modificación de la última capa completamente conectada
AlexNet_model1.classifier[6] = nn.Linear(1024,2)

In [ ]:
#Creación de instancia del dispositivo CUDA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Verificando si CUDA esta disponible
print(device)

#Si CUDA está disponible, se traslada el modelo a este dispositivo
AlexNet_model1.to(device)

cpu


AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(AlexNet_model1.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # obtener los inputs; "data" es una lista de [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()

        inputs = inputs.unsqueeze(1)

        # forward + backward + optimize
        output = AlexNet_model1(inputs.float())
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()

        # imprimir estadísticas
        running_loss += loss.item()
        if i % 150 == 149:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 150))
            running_loss = 0.0

print('Finished Training of AlexNet')

[1,   150] loss: 0.704
[1,   300] loss: 0.668
[1,   450] loss: 0.683
[2,   150] loss: 0.669
[2,   300] loss: 0.661
[2,   450] loss: 0.666
[3,   150] loss: 0.631
[3,   300] loss: 0.620
[3,   450] loss: 0.592
[4,   150] loss: 0.608
[4,   300] loss: 0.605
[4,   450] loss: 0.525
[5,   150] loss: 0.552
[5,   300] loss: 0.547
[5,   450] loss: 0.547
[6,   150] loss: 0.509
[6,   300] loss: 0.518
[6,   450] loss: 0.500
[7,   150] loss: 0.597
[7,   300] loss: 0.477
[7,   450] loss: 0.507
[8,   150] loss: 0.439
[8,   300] loss: 0.483
[8,   450] loss: 0.458
[9,   150] loss: 0.445
[9,   300] loss: 0.456
[9,   450] loss: 0.441
[10,   150] loss: 0.448
[10,   300] loss: 0.454
[10,   450] loss: 0.428
[11,   150] loss: 0.437
[11,   300] loss: 0.418
[11,   450] loss: 0.449
[12,   150] loss: 0.417
[12,   300] loss: 0.438
[12,   450] loss: 0.428
[13,   150] loss: 0.395
[13,   300] loss: 0.404
[13,   450] loss: 0.402
[14,   150] loss: 0.413
[14,   300] loss: 0.355
[14,   450] loss: 0.432
[15,   150] loss: 0

In [ ]:
#Testing Accuracy
correct = 0
total = 0
TP = 0
TN = 0
FN = 0
FP = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        images = images.unsqueeze(1)
        outputs = AlexNet_model1(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)

        for j in range(len(predicted)):
            if predicted[j] == labels[j] and predicted[j] == 0:
                TP += 1
            if predicted[j] == labels[j] and predicted[j] == 1:
                TN += 1
            if predicted[j] != labels[j] and predicted[j] == 0:
                FP += 1
            if predicted[j] != labels[j] and predicted[j] == 1:
                FN += 1
        
        correct += (predicted == labels).sum().item()

print('%d\t%d' % (TP, FN))
print('%d\t%d' % (FP, TN))
print('Accuracy test: %d%%' % (100 *(TP + TN) / (TP + TN + FN + FP)))
print('Precision test: %d%%' % (100 * TP / (TP + FP)))
print('Recall test: %d%%' % (100 * TP / (TP + FN)))

445	83
87	273
Accuracy test: 80%
Precision test: 83%
Recall test: 84%


In [ ]:
torch.save(AlexNet_model1, 'AlexNet_mispronounced_US_20epochs.pkl')

# 30 Epochs

In [ ]:
torch.hub._validate_not_a_forked_repo=lambda a,b,c: True
AlexNet_model2 = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

#Descripción del modelo
AlexNet_model2.eval()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
#Modificación de la primera capa convolucional
AlexNet_model2.features[0] = nn.Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

#Modificación de la penultima capa completamente conectada
AlexNet_model2.classifier[4] = nn.Linear(4096,1024)

#Modificación de la última capa completamente conectada
AlexNet_model2.classifier[6] = nn.Linear(1024,2)

In [ ]:
#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(AlexNet_model2.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # obtener los inputs; "data" es una lista de [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()

        inputs = inputs.unsqueeze(1)

        # forward + backward + optimize
        output = AlexNet_model2(inputs.float())
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()

        # imprimir estadísticas
        running_loss += loss.item()
        if i % 150 == 149:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 150))
            running_loss = 0.0

print('Finished Training of AlexNet')

[1,   150] loss: 0.702
[1,   300] loss: 0.682
[1,   450] loss: 0.673
[2,   150] loss: 0.675
[2,   300] loss: 0.665
[2,   450] loss: 0.658
[3,   150] loss: 0.644
[3,   300] loss: 0.591
[3,   450] loss: 0.653
[4,   150] loss: 0.667
[4,   300] loss: 0.644
[4,   450] loss: 0.632
[5,   150] loss: 0.571
[5,   300] loss: 0.590
[5,   450] loss: 0.608
[6,   150] loss: 0.501
[6,   300] loss: 0.588
[6,   450] loss: 0.547
[7,   150] loss: 0.471
[7,   300] loss: 0.524
[7,   450] loss: 0.521
[8,   150] loss: 0.490
[8,   300] loss: 0.507
[8,   450] loss: 0.485
[9,   150] loss: 0.456
[9,   300] loss: 0.440
[9,   450] loss: 0.504
[10,   150] loss: 0.459
[10,   300] loss: 0.493
[10,   450] loss: 0.448
[11,   150] loss: 0.453
[11,   300] loss: 0.470
[11,   450] loss: 0.390
[12,   150] loss: 0.462
[12,   300] loss: 0.437
[12,   450] loss: 0.407
[13,   150] loss: 0.423
[13,   300] loss: 0.410
[13,   450] loss: 0.390
[14,   150] loss: 0.394
[14,   300] loss: 0.446
[14,   450] loss: 0.414
[15,   150] loss: 0

In [ ]:
#Testing Accuracy
correct = 0
total = 0
TP = 0
TN = 0
FN = 0
FP = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data[0].to(device), data[1].to(device)
        images = images.unsqueeze(1)
        outputs = AlexNet_model2(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)

        for j in range(len(predicted)):
            if predicted[j] == labels[j] and predicted[j] == 0:
                TP += 1
            if predicted[j] == labels[j] and predicted[j] == 1:
                TN += 1
            if predicted[j] != labels[j] and predicted[j] == 0:
                FP += 1
            if predicted[j] != labels[j] and predicted[j] == 1:
                FN += 1
        
        correct += (predicted == labels).sum().item()

print('%d\t%d' % (TP, FN))
print('%d\t%d' % (FP, TN))
print('Accuracy test %d fold: %d%%' % (i, 100 * correct / total))
print('Accuracy test %d fold: %d%%' % (i, 100 *(TP + TN) / (TP + TN + FN + FP)))
print('Precision test %d fold: %d%%' % (i, 100 * TP / (TP + FP)))
print('Recall test %d fold: %d%%' % (i, 100 * TP / (TP + FN)))

456	72
102	258
Accuracy test 553 fold: 80%
Accuracy test 553 fold: 80%
Precision test 553 fold: 81%
Recall test 553 fold: 86%


In [ ]:
torch.save(AlexNet_model2, 'AlexNet_model.pkl')

In [ ]:
model = torch.load('AlexNet_model.pkl')